---

Reconecte ao Drive.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
DRIVE_DIRECTORY = "curso_ml"
DRIVE_DIRECTORY = os.path.join("/content/drive/MyDrive", DRIVE_DIRECTORY)

---

In [8]:
import pickle
import numpy as np

## Combinação de classificadores

Mais uma vez, vamos começar carregando os dados de treino e teste.

In [9]:
with open(os.path.join(DRIVE_DIRECTORY, "cover_type.pkl"), "rb") as f:
    (X_train, y_train, X_test, y_test) = pickle.load(f)

Também carregue os classificadores salvos no último exercício.

In [10]:
tree = pickle.load(open(os.path.join(DRIVE_DIRECTORY, "tree.sav"), 'rb'))
rf = pickle.load(open(os.path.join(DRIVE_DIRECTORY, "rf.sav"), 'rb'))
knn = pickle.load(open(os.path.join(DRIVE_DIRECTORY, "knn.sav"), 'rb'))
lr = pickle.load(open(os.path.join(DRIVE_DIRECTORY, "lr.sav"), 'rb'))
svm = pickle.load(open(os.path.join(DRIVE_DIRECTORY, "svm.sav"), 'rb'))
neural_net = pickle.load(open(os.path.join(DRIVE_DIRECTORY, "neural_net.sav"), 'rb'))

Separe o registro de índice 2 de `X_test` em uma variável `new_instance`, aplicando o método `reshape` para que o array resultante seja representado em `1` linha e o número de colunas igual ao número de atributos (você pode usar `-1` para o NumPy determinar este número sozinho). Exiba `new_instance` e seu `shape`.

In [11]:
new_instance = X_test[2].reshape(1, -1)
new_instance, new_instance.shape

(array([[-2.42751612, -0.35810696,  2.39062347, -0.83105729,  0.0410722 ,
         -1.2884323 ,  1.55697188, -1.6248467 , -2.70414875, -0.11760965,
          0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ]]),
 (1, 53))

Agora utilize `new_instance` para obter predições dos algoritmos carregados.

In [12]:
y_pred_tree = tree.predict(new_instance)
y_pred_rf = rf.predict(new_instance)
y_pred_knn = knn.predict(new_instance)
y_pred_lr = lr.predict(new_instance)
y_pred_svm = svm.predict(new_instance)
y_pred_nn = neural_net.predict(new_instance)

Exiba os resultados da predições.

In [13]:
y_pred_tree[0], y_pred_rf[0], y_pred_knn[0], y_pred_lr[0], y_pred_svm[0], y_pred_nn[0]

(np.int64(1), np.int64(1), np.int64(1), np.int64(5), np.int64(5), np.int64(1))

Como este é um problema com mais de uma classe, a lógica para determinar a classe vencedora é um pouco diferente. Vamos começar colocando os resultados em uma lista.

In [14]:
results = [y_pred_tree[0], y_pred_rf[0], y_pred_knn[0], y_pred_lr[0], y_pred_svm[0], y_pred_nn[0]]

In [15]:
results

[np.int64(1), np.int64(1), np.int64(1), np.int64(5), np.int64(5), np.int64(1)]

Agora, utilizando a função `np.unique`, vamos determinar quais são as classes preditas e quantas vezes elas foram preditas.

In [16]:
unique, counts = np.unique(results, return_counts=True)
unique, counts

(array([1, 5]), array([4, 2]))

Agora é preciso verificar qual a classe que apareceu mais vezes. Primeiro vamos determinar qual a contagem máxima.

In [17]:
max_counts = counts.max()
max_counts

np.int64(4)

E agora nós conseguimos filtrar as classes que mais apareceram como sendo aquelas cuja contagem é igual á contagem máxima.

In [18]:
top_classes = unique[counts == max_counts]
top_classes

array([1])

Finalmente, a lógica vai utilizar `top_classes` para exibir os resultados.

In [19]:
if len(top_classes) == 1:
    print("Classe encontrada:", top_classes[0])
else:
    print("Empate entre as classes:", top_classes)

Classe encontrada: 1


## Rejeição de classificadores

Utilize o método `predict_proba` para determinar as probabilidades encontradas pelos classificadores. Entretanto, você precisa deixar o algoritmo SVM de fora, porque da forma que nós o implementamos, ele não retorna valores de probabilidade.

In [20]:
y_prob_tree = tree.predict_proba(new_instance)
y_prob_rf = rf.predict_proba(new_instance)
y_prob_knn = knn.predict_proba(new_instance)
y_prob_lr = lr.predict_proba(new_instance)
y_prob_nn = neural_net.predict_proba(new_instance)

y_prob_tree, y_prob_rf, y_prob_knn, y_prob_lr, y_prob_nn

(array([[0.        , 0.66666667, 0.        , 0.        , 0.        ,
         0.33333333, 0.        ]]),
 array([[0.01, 0.73, 0.02, 0.  , 0.02, 0.21, 0.01]]),
 array([[0.        , 0.66666667, 0.        , 0.        , 0.        ,
         0.33333333, 0.        ]]),
 array([[4.21860316e-04, 4.75359643e-01, 2.72792312e-02, 5.68466193e-11,
         5.11236594e-03, 4.91811101e-01, 1.57981533e-05]]),
 array([[5.37235920e-09, 8.57635895e-01, 1.01532530e-01, 2.28985500e-18,
         9.96025320e-05, 4.07310825e-02, 8.84721683e-07]]))

Determine a confiança dos classificadores, sendo igual à probabilidade máxima atribuída.

In [21]:
confidence_tree = y_prob_tree.max()
confidence_rf = y_prob_rf.max()
confidence_knn = y_prob_knn.max()
confidence_lr = y_prob_lr.max()
confidence_nn = y_prob_nn.max()

confidence_tree, confidence_rf, confidence_knn, confidence_lr, confidence_nn

(np.float64(0.6666666666666666),
 np.float64(0.73),
 np.float64(0.6666666666666666),
 np.float64(0.4918111010861337),
 np.float64(0.8576358946075474))

A lógica em si é parecida com a do problema binário, mas primeiro nós filtramos para manter somente os resultados maiores que a confiança mínima. Depois, aplicamos o mesmo código mostrado na combinação de classificadores.

Utilize uma confiança mínima um pouco mais baixa, já que os resultados encontrados acima são menores. Isso é comum em problemas de classificação com mais de duas classes.

In [22]:
min_confidence = 0.75
n_algorithms = 0
results = []

if confidence_tree >= min_confidence:
    n_algorithms += 1
    results.append(y_pred_tree[0])

if confidence_rf >= min_confidence:
    n_algorithms += 1
    results.append(y_pred_rf[0])

if confidence_knn >= min_confidence:
    n_algorithms += 1
    results.append(y_pred_knn[0])

if confidence_lr >= min_confidence:
    n_algorithms += 1
    results.append(y_pred_lr[0])

if confidence_nn >= min_confidence:
    n_algorithms += 1
    results.append(y_pred_nn[0])

unique, counts = np.unique(results, return_counts=True)
max_counts = counts.max()
top_classes = unique[counts == max_counts]

if len(top_classes) == 1:
    print(f"Classe encontrada: {top_classes[0]}, baseado em {n_algorithms} algoritmos.")
else:
    print(f"Empate entre as classes: {top_classes}, baseado em {n_algorithms} algoritmos.")

Classe encontrada: 1, baseado em 1 algoritmos.
